In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd
from main_functions import *
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interactive_output, Output
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import pandas as pd
from tqdm import tqdm
import multiprocessing
multiprocessing.set_start_method("spawn", force=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dill
dill.settings['recurse'] = True

from main_functions import *



path = "Data/Historical_prices.xlsx"
df = pd.read_excel(path, sheet_name="data", header=[0, 1], index_col=0, parse_dates=True, skiprows=[2])
df = df.loc[:, pd.IndexSlice["XAU Curncy", :]]
df.columns = df.columns.droplevel(0)
df.columns = ["Close", "High", "Low", "Open"]

df.dropna(inplace=True)

In [ ]:

multipliers = [0.25, 0.5, 1.0, 1.25, 1.5, 2.0]
trender_sens_list = [1, 3, 7, 10]
modes = ["long_only", "long_short"]
models = ["GBM", "Jump-Diffusion", "Heston", "Bates"]

model_parameters = {
    "GBM": ["mu", "sigma"],
    "Jump-Diffusion": ["mu", "sigma", "lambda", "nu", "delta"],
    "Heston": ["mu", "kappa", "theta", "sigma_v", "rho", "v0"],
    "Bates": ["mu", "kappa", "theta", "sigma_v", "rho", "v0", "lambda", "nu", "delta"]
}

path = "Data/Historical_prices.xlsx"
df = pd.read_excel(path, sheet_name="data", header=[0, 1], index_col=0, parse_dates=True, skiprows=[2])
df = df.loc[:, pd.IndexSlice["XAU Curncy", :]]
df.columns = df.columns.droplevel(0)
df.columns = ["Close", "High", "Low", "Open"]
df.dropna(inplace=True)

gbm_params = calibrate_gbm(df)
jd_params = calibrate_jump_diffusion(df)
heston_params = calibrate_heston(df)
bates_params = calibrate_bates(df)

calibrated_params = {"GBM": gbm_params, "Jump-Diffusion": jd_params, "Heston": heston_params, "Bates": bates_params}


def simulate_strategy_config(model, param_name, multiplier, mode, trender_sens,df = df, calibrated_params = calibrated_params,
                             n_sims=500, n_days=252*5, steps_per_day=24, initial_capital=1000):

    params = calibrated_params[model]
    if model == "GBM":
        simulate_func = simulate_gbm_paths
    elif model == "Jump-Diffusion":
        simulate_func = simulate_jump_diffusion_paths
    elif model == "Heston":
        simulate_func = simulate_heston_paths
    elif model == "Bates":
        simulate_func = simulate_bates_paths
    else:
        raise ValueError("Unknown model")
    
    if param_name is not None:
        if param_name in params:
            params[param_name] = params[param_name] * multiplier
        else:
            raise ValueError(f"Parameter {param_name} not found for model {model}")
    
    S0 = df["Close"].iloc[-1]
    
    synthetic_prices = simulate_func(params, S0, n_days, n_sims)
    sigma_intraday = estimate_intraday_sigma(params, model_type=model, steps_per_day=steps_per_day)
    intraday_prices = brownian_bridge_paths(synthetic_prices, steps_per_day=steps_per_day, sigma_intraday=sigma_intraday)
    trender_results = apply_trender_to_simulations(intraday_prices, sensitivity=trender_sens)
    aggregated_results, individual_results, equity_curves = backtest_trender_multiple_paths(
        intraday_prices, trender_results, mode=mode, initial_capital=initial_capital
    )
    
    agg_df = pd.DataFrame(aggregated_results).T.drop(columns=["num_trades", "final_equity"])
    try:
        sharpe = float(agg_df.loc[:,"sharpe_ratio"].values[0])
    except Exception:
        sharpe = None
    try:
        annual_return = float(agg_df.loc[:,"annual_return"].values[0])
    except Exception:
        annual_return = None
    try:
        max_drawdown = float(agg_df.loc[:,"max_drawdown"].values[0])
    except Exception:
        max_drawdown = None
    try:
        win_ratio = float(agg_df.loc[:,"win_ratio"].values[0])
    except Exception:
        win_ratio = None
            
    return {
        "model": model,
        "parameter": param_name if param_name is not None else "baseline",
        "multiplier": multiplier,
        "mode": mode,
        "trender_sensitivity": trender_sens,
        "sharpe_ratio": sharpe,
        "annual_return": annual_return,
        "max_drawdown": max_drawdown,
        "win_ratio": win_ratio
    }

import __main__
__main__.simulate_strategy_config = simulate_strategy_config





tasks = []
for model in models:
    for mode in modes:
        for ts in trender_sens_list:
            tasks.append((model, None, 1.0, mode, ts))
            for param in model_parameters[model]:
                for m in multipliers:
                    tasks.append((model, param, m, mode, ts))


def run_task(task):
    try:
        return simulate_strategy_config(*task)
    except Exception as e:
        return {"error": str(e), "task": task}

from pathos.multiprocessing import ProcessingPool as Pool
pool = Pool()  
all_results = list(tqdm(pool.imap(run_task, tasks), total=len(tasks), desc="Processing tasks"))
pool.close()
pool.join()

baseline_dict = {}
for res in all_results:
    if "error" in res:
        print("Task error:", res["error"], "for task", res.get("task"))
        continue
    if res.get("parameter") == "baseline":
        key = (res["model"], res["mode"], res["trender_sensitivity"])
        baseline_dict[key] = res

summary_rows = []
for res in all_results:
    if "error" in res:
        continue
    if res.get("parameter") != "baseline":
        key = (res["model"], res["mode"], res["trender_sensitivity"])
        base_res = baseline_dict.get(key)
        if base_res is not None and base_res.get("sharpe_ratio") is not None and res.get("sharpe_ratio") is not None:
            base_sharpe = base_res["sharpe_ratio"]
            new_sharpe = res["sharpe_ratio"]
            diff = new_sharpe - base_sharpe
            characterization = "Improvement" if diff > 0 else "Pitfall" if diff < 0 else "No Change"
        else:
            base_sharpe = None
            new_sharpe = None
            diff = None
            characterization = "N/A"
        summary_rows.append({
            "model": res["model"],
            "parameter": res["parameter"],
            "multiplier": res["multiplier"],
            "mode": res["mode"],
            "trender_sensitivity": res["trender_sensitivity"],
            "baseline_sharpe": base_sharpe,
            "new_sharpe": new_sharpe,
            "difference": diff,
            "characterization": characterization
        })

summary_df = pd.DataFrame(summary_rows)
print("Summary of Grid Search Results:")
print(summary_df)
